In [2]:
import lib as lib
import basis_set as bs
import numpy as np
import os
import mc_integration as mc

### Hartree Fock computation of the Helium atom

First we set up the simulation

In [ ]:
N_electrons = 2
N_basis = 2
integrals_file = "integrals_He.csv"

normalized_wf = False

max_iter_SCF = 100
eps_SCF = 1E-5

Now we make minor modifications on the integral_master class to adapt it to the computation of the Helium atom, whose basis functions are slightly different.

In [ ]:
class integral_master_He():
	"""
	Calculates, stores and retrieves the values of the <pr|g|qs> integrals
	"""
	def __init__(self, dimension):
		"""
		Initialization of the object

		Parameters
		----------
		dimension : int
			Number of single basis functions

		Returns
		-------
		None
		"""

		self.integral_dict_1 = None
		self.integral_dict_2 = None
		self.dimension = dimension

		return

	def calculate(self, file_name):
		"""
		Calculates the <pr|g|qs> integrals and stores them in file

		Parameters
		----------
		file_name : str
			File name that stores the values of the integrals

		Returns
		-------
		None
		"""

		if file_name in os.listdir():
			print("Integral file already exsists. Not computing the integrals. ")
			return

		integral_dict_1 = {}
		integral_dict_2 = {}

		# 1-body integrals
		for p in range(1, self.dimension+1):
			for q in range(1, self.dimension+1):
				if p == q:
					I = self.calculate_1(p, q)
				else:
					I = 0

				integral_dict_1[(p, q)] = I

		# 2-body integrals
		for p in range(1, self.dimension+1):
			for q in range(1, p+1):
				for r in range(1, p):
					for s in range(1, r+1):
						I = self.calculate_2(p, r, q, s)

						integral_dict_2[(p, r, q, s)] = I
						integral_dict_2[(q, r, p, s)] = I
						integral_dict_2[(p, s, q, r)] = I
						integral_dict_2[(r, p, s, q)] = I
				r = p
				for s in range(1, q+1):
					I = self.calculate_2(p, r, q, s)
					
					integral_dict_2[(p, r, q, s)] = I
					integral_dict_2[(q, r, p, s)] = I
					integral_dict_2[(p, s, q, r)] = I
					integral_dict_2[(r, p, s, q)] = I

		np.save(file_name, np.array([integral_dict_1, integral_dict_2]))

		return

	def load_integrals(self, file_name):
		"""
		Loads the values of the integrals in this object

		Parameters
		----------
		file_name : str
			File name that stores the values of the integrals

		Returns
		-------
		None
		"""

		self.integral_dict_1, self.integral_dict_2 = np.load(file_name)

		return

	def get_1(self, p, q):
		"""
		Returns the value of the h_pq integrals from the class dictionaries

		Parameters
		----------
		p, q: int
			Indices that specify the h_pq integral

		Returns
		-------
		I : float
			Value of the h_pq integral
		"""

		I = self.integral_dict_1[(p, q)]

		return I

	def get_2(self, p, r, q, s):
		"""
		Returns the value of the <pr|g|qs> integrals from the class dictionaries

		Parameters
		----------
		p, r, q, s: int
			Indeces that specify the <pr|g|qs> integral

		Returns
		-------
		I : float
			Value of the <pr|g|qs> integral
		"""

		I = self.integral_dict_2[(p, r, q, s)]

		return I

		
	def calculate_1(self, p, q):
		"""
		Calculates the value of the h_pq integrals by direct integration

		Parameters
		----------
		p, q: int
			Indices that specify the h_pq integral

		Returns
		----------
		I : float
			Value of the h_pq integral
		"""
		nx,ny,nz = bs.index_to_q_numbers(p)

		I = (p==q)*(bs.OMEGA_X*(nx + ny + 1) + bs.OMEGA_Z*(nz + 0.5)) # nx,ny,nz should come from the basis, so from p and q but how?

		return I


	def calculate_2(self, p, r, q, s):
		"""
		Calculates the value of the <pr|g|qs> integrals by monte carlo integration methods

		Parameters
		----------
		p, q, r, s: int
			Indices that specify the <pr|g|qs> integral

		Returns
		----------
		I : float
			Value of the <pr|g|qs> integral
		"""

		system_size = 5
		N_walkers = 400
		N_steps = 10000
		N_skip = 1000
		integrand = bs.He_two_body_integrand
		indices = np.array([p,r,q,s])
		dimension = 6
		
		I = mc.MC_integration(integrand, indices, dimension, N_steps, N_walkers, N_skip, system_size)

		return I

### Computation block
In the next piece of code we run the actual computation of the energy levels of the Helium atom. First we initialize the class to compute the integrals using the four basis functions defined in Jos' book and then we initialize a random coeficient matrix and we compute the density matrix. From there we actually compute the matrix and then we check whether the wave functions are normalized and compute the matrix S accordingly.

In [ ]:
integrals = integral_master_He(4)
C = np.random.rand(N_basis, N_basis)
rho = lib.density_matrix(C, N_electrons)

	# One- and Two-body integrals
integrals.calculate(integrals_file)

	# Normalization of wave functions
if not normalized_wf:
	# calculate S matrix
	# (...)
	SVAL, SVEC = np.linalg.eigh(S) 
	SVAL_minhalf = (np.diag(SVAL**(-0.5))) 
	X = np.dot(SVEC, np.dot(SVAL_minhalf, np.transpose(SVEC)))
else:
	S = np.eye(N_basis)

Finally, we run the self consistent field iteration to recursively update C until a convergence condition is fulfilled.

In [ ]:
# Self Consistent Field
n_iterations = 0
rho_old = np.zeros((N_basis, N_basis))

while n_iterations < max_iter_SCF:
	n_iterations += 1

	F = lib.create_F_matrix(rho, integrals)
	
	if normalized_wf:
		E, C = np.linalg.eigh(F)
	else:
		F_prime = np.conjugate(X.transpose()) @ F @ X
		E, C_prime = np.linalg.eigh(F_prime)
		C = X @ C_prime
	
	rho = lib.density_matrix(C, N_electrons)
	total_E = lib.total_energy(rho, F, integrals)

	if lib.delta_rho(rho, rho_old) < eps_SCF:
		break
	
	total_E_old = total_E
	rho_old = rho

	print("E = {:0.7f} | N(SCF) = {}".format(total_E, n_iterations))

The next is just a check block in which one of the two body integrals is computed. We find that its divergent :(

In [4]:
from scipy import integrate
import numpy as np
import basis_set as bs
indices = np.array([2,2,1,1])
I = integrate.nquad(bs.He_two_body_integrand,[[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf]],args=indices,opts={"limit":100})

C:\Users\alvar\Documents\TUDelft\2. Computational Physics\Project3_abermejillo_dbedialaunetar_mserraperalta\basis_set.py:292: RuntimeWarning: divide by zero encountered in double_scalars
  I = He_wf_basis(R[0:3],p)*He_wf_basis(R[3:6],r)*(1/r12)*He_wf_basis(R[0:3],q)*He_wf_basis(R[3:6],s)
C:\Users\alvar\AppData\Roaming\Python\Python36\site-packages\scipy\integrate\quadpack.py:881: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  **opt)
C:\Users\alvar\AppData\Roaming\Python\Python36\site-packages\scipy\integrate\quadpack.py:881: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  **opt)
C:\Users\alvar\AppData\Roaming\Python\Python36\site-packages\scipy\integrate\quadpack.py:881: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) 

KeyboardInterrupt: 